In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
from pylab import *
from scipy.stats import spearmanr
from scipy.stats._stats import _kendall_dis
from matplotlib.colors import ListedColormap
import matplotlib.colors as clr
from google.colab import files

# The Vancouver Retrofit Survey Dataset

Here the final dataset for the survey is created. We compiled all of the responses into a csv file and will no encode the text for numerics, so that it can be analyzed more easily.

We also add several columns on for binary data realted to policy options chosen, as well as join the narrative parameters to the response dataset.

While the inital response dataset is presented here, some modification and quality control took place before finalizing it. 

## Policy Responses

For the response option "Please specify" to the question regarding policy interventions that would enhance the likelihood of retrofit occuring we made the decision to intepret, where possible, responses into either existing catageories or combined into new. The table bloew presents these transformations.

In [ ]:
# This is the dataframe for the altered policy responses
policyupdatesfile = r'https://raw.githubusercontent.com/justinfmccarty/forcolab/master/vrs/final/policy_updates.csv'

dfpolupdates = pd.DataFrame(pd.read_csv(policyupdatesfile))
dfpolupdates


,id,intial response,chosen category,existing or new?
0,24gwh1,Heritage financing (i.e. helping homeowners wh...,Innovative financing mechanisms such as attach...,existing
1,23vs6,Crown Corporation or other powerful public ent...,The creation of an integrated public retrofit ...,existing
2,23mp3,Crown Corporation or other powerful public ent...,The creation of an integrated public retrofit ...,existing
3,22mp4,Utility offer of owning and renting heat pump ...,Innovative financing mechanisms such as attach...,existing
4,25vs4,"Retrofit energy codes are a waste of time, nob...",Other,new
5,26vs5,You need to incentivise based on metered savin...,Innovative financing mechanisms such as attach...,existing
6,17sb2,Land use adaptation (multi-generational housin...,Incentivise retofits with suite/ADU densification,new
7,16vs4,"A comprehensive, multi-level financing scheme ...",Innovative financing mechanisms such as attach...,existing
8,16sb3,I do not believe this house will be retrofitted.,No chance,new
9,17sb4,Unless there is such a comprehensive financing...,Innovative financing mechanisms such as attach...,existing


## Response Dataset

In [ ]:
# This is the dataframe for the survey responses
file = r'https://raw.githubusercontent.com/justinfmccarty/forcolab/master/vrs/final/final_raw_responses_text.csv'

df_responses_text = pd.DataFrame(pd.read_csv(file))
df_responses_text.index=df_responses_text['unique_id']
df_responses_text_sub = df_responses_text[['optimism_int','resp-additional_info','resp-influential_param']]
df_responses_text.head(3)

,survey number,all narratives,archetype,optimism_str,optimism_int,id,unique_id,expqual-field,expqual-field_notes,expqual-field_years,expqual-step_familiar,expqual-market_familiar,resp-likert_step2,resp-likert_step3,resp-likert_step4,resp-likert_step5,resp-likert_higheff,resp-retrofit_teardown_neither,resp-likert_carbon2050,resp-influential_param,resp-policy1,resp-policy2,resp-policy3,resp-confidence,resp-additional_info
unique_id,,,,,,,,,,,,,,,,,,,,,,,,,
29mp6,29,"MP-vo, SB-sp, WC-p",mp,vo,6,mp6,29mp6,Government and public works,0,15 to 19,Very unfamiliar,Familiar,somewhat likely,neutral,somewhat unlikely,very unlikely,unlikely,more likely to be demolished and rebuilt,likely,If this house were outside of Vancouver it wou...,an energy disclosure mandate with a benchmarki...,innovative financing mechanisms such as attach...,the implementation of a retrofit code for ener...,medium confidence,no response
29sb3,29,"MP-vo, SB-sp, WC-p",sb,sp,3,sb3,29sb3,Government and public works,0,15 to 19,Very unfamiliar,Familiar,unlikely,somewhat unlikely,very unlikely,very unlikely,somewhat unlikely,more likely to be demolished and rebuilt,unlikely,Even relatively new homes like this often end ...,an energy disclosure mandate with a benchmarki...,innovative financing mechanisms such as attach...,the implementation of a retrofit code for ener...,medium confidence,no response
29wc2,29,"MP-vo, SB-sp, WC-p",wc,p,2,wc2,29wc2,Government and public works,0,15 to 19,Very unfamiliar,Familiar,very likely,somewhat unlikely,very unlikely,very unlikely,neutral,"neither (the building will exist, but will not...",very likely,Assuming the electricity grid continues to mov...,an energy disclosure mandate with a benchmarki...,an education campaign for owners and practitio...,the implementation of a retrofit code for ener...,high confidence,no response


In [ ]:
# encoding definitions for responses

archetype = {'mp':1,
             'sb':2,
             'wc':3,
             'vs':4,
             'gwh':5}
field = {'Architecture, planning, and environmental design': 1,
         'Business and financial services': 2,
         'Communications': 3,
         'Education': 4,
         'Engineering and computer science': 5,
         'Environment': 6,'Government and public works': 7,
         'Law and public policy': 8,
         'Community and social service': 9,
         'Other': 10}

fieldyears = {'less than 5':1,
              '5 to 9':2,
              '10 to 14':3,
              '15 to 19':4,
              '20 to 24':5,
              '25 or more':6}

familiar = {'Very unfamiliar':1,
           'Unfamiliar':2,
           'Slightly unfamiliar':3,
           'Neutral':4,
           'Slightly familiar':5,
           'Familiar':6,
           'Very familiar':7}

confidence = {'very low confidence':1,
              'low confidence':2,
              'medium confidence':3,
              'high confidence':4,
              'very high confidence':5}

likert = {'very unlikely':1,
          'somewhat unlikely':2,
          'unlikely':3,
          'neutral':4,
          'likely':5,
          'somewhat likely':6,
          'very likely':7}

rtn = {'more likely to retrofit to step code 5':1,
       'more likely to be demolished and rebuilt':2,
       'neither (the building will exist, but will not achieve step code 5 by 2050)':3}

policy = {'an energy disclosure mandate with a benchmarking, audit, and labeling protocol':1,
          'an education campaign for owners and practitioners on potential savings and best practices':4,
          'aggregation of individual retrofit projects into packages':3,
          'innovative financing mechanisms such as attaching loans to meter savings':6,
          "a public financing scheme such as a provincial 'green bank'":2,
          'no chance':10,
          'conventional financing mechanism such as grants or tax abatements':5,
          'the creation of an integrated public retrofit servicing agency':7,
          'incentivise retofits with suite/adu densification':9,
          'other':11,
          'the implementation of a retrofit code for energy, emissions, and safety upgrades':8}




In [ ]:
df_responses_numeric = pd.DataFrame(df_responses_text[['archetype',
                                                        'expqual-field',
                                                        'expqual-field_years',
                                                        'expqual-step_familiar',
                                                        'expqual-market_familiar',
                                                        'resp-likert_step2',
                                                        'resp-likert_step3',
                                                        'resp-likert_step4',
                                                        'resp-likert_step5',
                                                        'resp-likert_higheff',
                                                        'resp-retrofit_teardown_neither',
                                                        'resp-likert_carbon2050',
                                                        'resp-policy1',
                                                        'resp-policy2',
                                                        'resp-policy3',
                                                        'resp-confidence']])


df_responses_numeric = df_responses_numeric.replace({'archetype':archetype,
                                                      'expqual-field':field,
                                                      'expqual-field_years':fieldyears,
                                                      'expqual-step_familiar':familiar,
                                                      'expqual-market_familiar':familiar,
                                                      'resp-likert_step2':likert,
                                                      'resp-likert_step3':likert,
                                                      'resp-likert_step4':likert,
                                                      'resp-likert_step5':likert,
                                                      'resp-likert_higheff':likert,
                                                      'resp-retrofit_teardown_neither':rtn,
                                                      'resp-likert_carbon2050':likert,
                                                      'resp-policy1':policy,
                                                      'resp-policy2':policy,
                                                      'resp-policy3':policy,
                                                      'resp-confidence':confidence}).astype(int)

In [ ]:
# need to create columns tha trepresent each policy option and 
#a binary indication of whether or not it was responded to 

pol1 = pd.DataFrame(df_responses_numeric[['resp-policy1']])
pol2 = pd.DataFrame(df_responses_numeric[['resp-policy2']])
pol3 = pd.DataFrame(df_responses_numeric[['resp-policy3']])

def policy(x,number):
  if x == number:
    return 1
  else:
    return 0

pol1['binary-policyopt1'] = pol1['resp-policy1'].apply(lambda x: policy(x,1))
pol1['binary-policyopt2'] = pol1['resp-policy1'].apply(lambda x: policy(x,2))
pol1['binary-policyopt3'] = pol1['resp-policy1'].apply(lambda x: policy(x,3))
pol1['binary-policyopt4'] = pol1['resp-policy1'].apply(lambda x: policy(x,4))
pol1['binary-policyopt5'] = pol1['resp-policy1'].apply(lambda x: policy(x,5))
pol1['binary-policyopt6'] = pol1['resp-policy1'].apply(lambda x: policy(x,6))
pol1['binary-policyopt7'] = pol1['resp-policy1'].apply(lambda x: policy(x,7))
pol1['binary-policyopt8'] = pol1['resp-policy1'].apply(lambda x: policy(x,8))
pol1['binary-policyopt9'] = pol1['resp-policy1'].apply(lambda x: policy(x,9))
pol1['binary-policyopt10'] = pol1['resp-policy1'].apply(lambda x: policy(x,10))
pol1['binary-policyopt11'] = pol1['resp-policy1'].apply(lambda x: policy(x,11))
del pol1['resp-policy1']


pol2['binary-policyopt1'] = pol2['resp-policy2'].apply(lambda x: policy(x,1))
pol2['binary-policyopt2'] = pol2['resp-policy2'].apply(lambda x: policy(x,2))
pol2['binary-policyopt3'] = pol2['resp-policy2'].apply(lambda x: policy(x,3))
pol2['binary-policyopt4'] = pol2['resp-policy2'].apply(lambda x: policy(x,4))
pol2['binary-policyopt5'] = pol2['resp-policy2'].apply(lambda x: policy(x,5))
pol2['binary-policyopt6'] = pol2['resp-policy2'].apply(lambda x: policy(x,6))
pol2['binary-policyopt7'] = pol2['resp-policy2'].apply(lambda x: policy(x,7))
pol2['binary-policyopt8'] = pol2['resp-policy2'].apply(lambda x: policy(x,8))
pol2['binary-policyopt9'] = pol2['resp-policy2'].apply(lambda x: policy(x,9))
pol2['binary-policyopt10'] = pol2['resp-policy2'].apply(lambda x: policy(x,10))
pol2['binary-policyopt11'] = pol2['resp-policy2'].apply(lambda x: policy(x,11))
del pol2['resp-policy2']


pol3['binary-policyopt1'] = pol3['resp-policy3'].apply(lambda x: policy(x,1))
pol3['binary-policyopt2'] = pol3['resp-policy3'].apply(lambda x: policy(x,2))
pol3['binary-policyopt3'] = pol3['resp-policy3'].apply(lambda x: policy(x,3))
pol3['binary-policyopt4'] = pol3['resp-policy3'].apply(lambda x: policy(x,4))
pol3['binary-policyopt5'] = pol3['resp-policy3'].apply(lambda x: policy(x,5))
pol3['binary-policyopt6'] = pol3['resp-policy3'].apply(lambda x: policy(x,6))
pol3['binary-policyopt7'] = pol3['resp-policy3'].apply(lambda x: policy(x,7))
pol3['binary-policyopt8'] = pol3['resp-policy3'].apply(lambda x: policy(x,8))
pol3['binary-policyopt9'] = pol3['resp-policy3'].apply(lambda x: policy(x,9))
pol3['binary-policyopt10'] = pol3['resp-policy3'].apply(lambda x: policy(x,10))
pol3['binary-policyopt11'] = pol3['resp-policy3'].apply(lambda x: policy(x,11))
del pol3['resp-policy3']


binarypols = np.clip(pol1+pol2+pol3,0,1)
binarypols.head(3)


,binary-policyopt1,binary-policyopt2,binary-policyopt3,binary-policyopt4,binary-policyopt5,binary-policyopt6,binary-policyopt7,binary-policyopt8,binary-policyopt9,binary-policyopt10,binary-policyopt11
unique_id,,,,,,,,,,,
29mp6,1,0,0,0,0,1,0,1,0,0,0
29sb3,1,0,0,0,0,1,0,1,0,0,0
29wc2,1,0,0,1,0,0,0,1,0,0,0


In [ ]:
# finally we concatenate the numeric responses, the binary policy table, and
# the long form text responses

df_responses_whole = pd.concat([df_responses_numeric,binarypols,df_responses_text_sub],axis=1,join="outer")


## Narrative Element Dataset

In [ ]:
# This is the dataframe for the survey responses
fileelements = r'https://raw.githubusercontent.com/justinfmccarty/forcolab/master/vrs/final/final_narrative_elements_text.csv'

df_elements_text = pd.DataFrame(pd.read_csv(fileelements))
#df_elements_text.index=df_elements_text['unique_id']

df_elements_text.head(3)

,archetype,optimism,id,narr-age,narr-family type,narr-duration,narr-spending,narr-env,narr-env_note,narr-income,narr-debt_note,narr-mortgage percent,narr-credit,prm-house sqft,prm-lot sqft,prm-house age,prm-step,prm-suite,prm-reno_plan,prm-wall_type,prm-wall_age,prm-window_type,prm-window_age,prm-roof_type,prm-roof_age,prm-heat_eff,prm-heat_fuel,prm-heat_age,prm-dhw_eff,prm-dhw_fuel,prm-dhw_age,prm-cool_eff,prm-cool_fuel,prm-cool_age,prm-leakage,prm-maintenance,prm-asbestos
0,vs,1,vs1,75-84,couple,34,frugal,some,NaN,retired,none mentioned,0,none mentioned,2600,4000,44,1,no,no,2x4 batt R13,19,single 0.48,19,blown R15,19,0.85,ng,8,0.85,ng,10,none,none,none,leaky,average,yes
1,vs,2,vs2,45-54,couple,5,thrifty,none,NaN,below average,none mentioned,10,good,2600,4000,44,1,no,no,2x4 batt R13,19,single 0.48,19,blown R15,19,0.85,ng,29,0.85,ng,8,none,none,none,leaky,never,no
2,vs,3,vs3,45-54,couple,5,thrifty,none,NaN,average,none mentioned,10,good,2600,4000,44,1,no,no,2x4 batt R13,19,single 0.48,19,blown R15,19,0.85,ng,8,0.85,ng,8,none,none,none,leaky,average,no


In [ ]:
print(df_elements_text.columns)

Index(['archetype', 'optimism', 'id', 'narr-age', 'narr-family type',
       'narr-duration', 'narr-spending', 'narr-env', 'narr-env_note',
       'narr-income', 'narr-debt_note', 'narr-mortgage percent', 'narr-credit',
       'prm-house sqft', 'prm-lot sqft', 'prm-house age', 'prm-step',
       'prm-suite', 'prm-reno_plan', 'prm-wall_type', 'prm-wall_age',
       'prm-window_type', 'prm-window_age', 'prm-roof_type', 'prm-roof_age',
       'prm-heat_eff', 'prm-heat_fuel', 'prm-heat_age', 'prm-dhw_eff',
       'prm-dhw_fuel', 'prm-dhw_age', 'prm-cool_eff', 'prm-cool_fuel',
       'prm-cool_age', 'prm-leakage', 'prm-maintenance', 'prm-asbestos'],
      dtype='object')


In [ ]:
cols = df_elements_text[['archetype',
                          'id',
                          'narr-age',
                          'narr-family type',
                          'narr-duration', 
                          'narr-spending', 
                          'narr-env', 
                          'narr-income', 
                          'narr-mortgage percent',
                          'narr-credit',
                          'prm-house sqft', 
                          'prm-lot sqft', 
                          'prm-house age', 
                          'prm-step',
                          'prm-suite', 
                          'prm-reno_plan',
                          'prm-wall_type',
                          'prm-wall_age',
                          'prm-window_type', 
                          'prm-window_age',
                          'prm-roof_type',
                          'prm-roof_age',
                          'prm-heat_eff', 
                          'prm-heat_fuel',
                          'prm-heat_age', 
                          'prm-dhw_eff',
                          'prm-dhw_fuel', 
                          'prm-dhw_age', 
                          'prm-cool_eff',
                          'prm-cool_fuel',
                          'prm-cool_age', 
                          'prm-leakage', 
                          'prm-maintenance', 
                          'prm-asbestos']]


In [ ]:
#'archetype',
#'optimism',
#'id',
narr_age = {'25-34':1,
            '35-44':2,
            '45-54':3,
            '55-64':4,
            '65-74':5,
            '75-84':6}
            
narr_family_type = {'couple':1,
                    'couple+kids':2,
                    'couple+kids+other':3,
                    'single':4,
                    'single+kids':5}

def narr_duration(x):
  if 0 <= x <= 7:
    return 1
  elif 8 <= x <= 14:
    return 2
  elif 15 <= x <= 21:
    return 3
  elif 22 <= x <= 28:
    return 4
  elif x >= 29:
    return 5

narr_spending = {'frugal':1,
                  'thrifty':2,
                  'generous':3}

narr_env = {'none':1,
            'some':2,
            'intense':3}

narr_income = {
               'below average':1,
                'average':2,
                'above average':3,
                'retired':4}

def narr_mortgage_percent(x):
  if 0 <= x <= 8:
   return 1
  elif 9 <= x <= 16:
    return 2
  elif 17 <= x <= 24:
   return 3
  elif 25 <= x <= 32:
   return 4
  elif x >= 33:
   return 5

narr_credit = {'good':1,
                'fair':2,
                'excellent':3,
                'none mentioned':9}

prm_house_sqft = {'2100':1,
                  '2600':2,
                  '2800':3,
                  '3000':4,
                  '5400':5}

prm_lot_sqft = {'3135':1,
                '4000':2,
                '8400':3}


prm_house_age = {'108':1,
                  '67':2,
                  '44':3,
                  '28':4,
                  '1':5}

prm_step = {'below 1':0,
            '1':1,
            '2':2}

prm_suite = {'no':0,
            'yes':1}

prm_reno_plan = {'no':0,
                  'yes':1}

prm_wall_type = {'2x4 batt R13':1,
                 '2x6 ext R21':2}

def prm_ages(x):
  if 0 <= x <= 5:
   return 1
  elif 6 <= x <= 10:
    return 2
  elif 11 <= x <= 15:
   return 3
  elif 16 <= x <= 24:
   return 4
  elif x >= 25:
   return 5

prm_cool_age = {'none':0,
                '1':1}

prm_window_type = {'double 0.25':1,
                   'double 0.48':1,
                   'single 0.48':2}

prm_roof_type = {'blown R15':1,
                 'flat ext R36':2}

prm_heat_eff = {'0.75':1,
                '0.85':2,
                '2.7':3} 
def fuel(x):
  if x == 'ng':
    return 1
  elif x == 'electric':
    return 2
  elif x == r'ng/electric':
    return 3
  elif x == 'none':
    return 0



prm_dhw_eff = {'0.75':1,
                '0.85':2, 
                '0.95':3}

prm_cool_eff = {'none':0,
                '2.7':1}


prm_leakage = {'very leaky':1,
                'leaky':2,
                'middle':3,
                'tight':4}

prm_maintenance = {'never':1,
                    'poor':2,
                    'average':3,
                    'well maintained':4,
                    'very well':5}
prm_asbestos = {'no':0,
                'yes':1}

In [ ]:
df_elements_numeric = pd.DataFrame(cols).astype(str)
df_elements_numeric = df_elements_numeric.replace({'archetype':archetype,
                                                      'narr-age':narr_age,
                                                      'narr-family type':narr_family_type,
                                                      'narr-spending':narr_spending,
                                                      'narr-env':narr_env,
                                                      'narr-income':narr_income,
                                                      'narr-credit':narr_credit,
                                                      'prm-house sqft':prm_house_sqft,
                                                      'prm-lot sqft':prm_lot_sqft,
                                                      'prm-house age':prm_house_age,
                                                      'prm-step':prm_step,
                                                      'prm-suite':prm_suite,
                                                      'prm-reno_plan':prm_reno_plan,
                                                      'prm-wall_type':prm_wall_type,
                                                      'prm-window_type':prm_window_type,
                                                      'prm-roof_type':prm_roof_type,
                                                      'prm-heat_eff':prm_heat_eff,
                                                      'prm-dhw_eff':prm_dhw_eff,
                                                      'prm-cool_eff':prm_cool_eff,
                                                      'prm-cool_age':prm_cool_age,
                                                      'prm-leakage':prm_leakage,
                                                      'prm-maintenance':prm_maintenance,
                                                      'prm-asbestos':prm_asbestos})
df_elements_numeric['narr-duration']=df_elements_numeric['narr-duration'].astype(int)
df_elements_numeric['narr-duration']=df_elements_numeric['narr-duration'].apply(lambda x: narr_duration(x))

df_elements_numeric['narr-mortgage percent']=df_elements_numeric['narr-mortgage percent'].astype(int)
df_elements_numeric['narr-mortgage percent']=df_elements_numeric['narr-mortgage percent'].apply(lambda x: narr_mortgage_percent(x))

df_elements_numeric['prm-wall_age']=df_elements_numeric['prm-wall_age'].astype(int)
df_elements_numeric['prm-window_age']=df_elements_numeric['prm-window_age'].astype(int)
df_elements_numeric['prm-roof_age']=df_elements_numeric['prm-roof_age'].astype(int)
df_elements_numeric['prm-heat_age']=df_elements_numeric['prm-heat_age'].astype(int)
df_elements_numeric['prm-dhw_age']=df_elements_numeric['prm-dhw_age'].astype(int)

df_elements_numeric['prm-wall_age']=df_elements_numeric['prm-wall_age'].apply(lambda x: prm_ages(x))
df_elements_numeric['prm-window_age']=df_elements_numeric['prm-window_age'].apply(lambda x: prm_ages(x))
df_elements_numeric['prm-roof_age']=df_elements_numeric['prm-roof_age'].apply(lambda x: prm_ages(x))
df_elements_numeric['prm-heat_age']=df_elements_numeric['prm-heat_age'].apply(lambda x: prm_ages(x))
df_elements_numeric['prm-dhw_age']=df_elements_numeric['prm-dhw_age'].apply(lambda x: prm_ages(x))



df_elements_numeric['prm-heat_fuel']=df_elements_numeric['prm-heat_fuel'].apply(lambda x: fuel(x))
df_elements_numeric['prm-dhw_fuel']=df_elements_numeric['prm-dhw_fuel'].apply(lambda x: fuel(x))
df_elements_numeric['prm-cool_fuel']=df_elements_numeric['prm-cool_fuel'].apply(lambda x: fuel(x))


df_elements_numeric['prm_derived_FAR'] = df_elements_text['prm-house sqft'] / df_elements_text['prm-lot sqft']

In [ ]:
df_elements_numeric.shape

(30, 35)

## Join Responses and Narrative Elements

In [ ]:
# if df_responses_whole.index string contains df_responses_numeric['id'] then join

# need to first bring out the index into a id only column
df_responses_whole['id'] = df_responses_whole.index
df_responses_whole['unique_id'] = df_responses_whole.index
resp = df_responses_whole
elem = df_elements_numeric
result = resp['id'].str.split('([A-Za-z]+\d+)', expand=True)
resp['respondent']= result.loc[:,[0]]
resp['id'] = result.loc[:,[1]]
resp['id_check']=resp['id']

# now merge on id
all = pd.merge(resp,elem,how='outer',on='id')
check = all['id'] == all['id_check']

# drop any narrative that was added that was not responded to in the survey
all = all.drop(all[all['id'] != all['id_check']].index)
all.index = all['unique_id']


In [ ]:
all.head()

,archetype_x,expqual-field,expqual-field_years,expqual-step_familiar,expqual-market_familiar,resp-likert_step2,resp-likert_step3,resp-likert_step4,resp-likert_step5,resp-likert_higheff,resp-retrofit_teardown_neither,resp-likert_carbon2050,resp-policy1,resp-policy2,resp-policy3,resp-confidence,binary-policyopt1,binary-policyopt2,binary-policyopt3,binary-policyopt4,binary-policyopt5,binary-policyopt6,binary-policyopt7,binary-policyopt8,binary-policyopt9,binary-policyopt10,binary-policyopt11,optimism_int,resp-additional_info,resp-influential_param,id,unique_id,respondent,id_check,archetype_y,narr-age,narr-family type,narr-duration,narr-spending,narr-env,narr-income,narr-mortgage percent,narr-credit,prm-house sqft,prm-lot sqft,prm-house age,prm-step,prm-suite,prm-reno_plan,prm-wall_type,prm-wall_age,prm-window_type,prm-window_age,prm-roof_type,prm-roof_age,prm-heat_eff,prm-heat_fuel,prm-heat_age,prm-dhw_eff,prm-dhw_fuel,prm-dhw_age,prm-cool_eff,prm-cool_fuel,prm-cool_age,prm-leakage,prm-maintenance,prm-asbestos,prm_derived_FAR
unique_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
29mp6,1.0,7.0,4.0,1.0,6.0,6.0,4.0,2.0,1.0,3.0,2.0,5.0,1.0,6.0,8.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,6.0,no response,If this house were outside of Vancouver it wou...,mp6,29mp6,29,mp6,1,4,1,4,1,3,2,1,1,1,2,2,1,0,0,1,3,1,5,1,5,1,1,5,1,1,5,0,0,0,2,3,0,0.525000
7mp6,1.0,7.0,1.0,6.0,3.0,5.0,6.0,2.0,1.0,3.0,2.0,1.0,1.0,2.0,8.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,no response,"Energy Step Code Metrics, Vancouver single fam...",mp6,7mp6,7,mp6,1,4,1,4,1,3,2,1,1,1,2,2,1,0,0,1,3,1,5,1,5,1,1,5,1,1,5,0,0,0,2,3,0,0.525000
29sb3,2.0,7.0,4.0,1.0,6.0,3.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0,6.0,8.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,no response,Even relatively new homes like this often end ...,sb3,29sb3,29,sb3,2,6,1,4,1,1,3,1,9,5,3,4,1,0,0,1,5,2,5,1,5,1,1,5,3,2,1,0,0,0,3,3,0,0.642857
16sb3,2.0,10.0,3.0,7.0,6.0,5.0,5.0,3.0,2.0,5.0,2.0,6.0,10.0,6.0,11.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,This type of house is in a 'dead zone' of retr...,"Age of the homeowners, and likelihood that new...",sb3,16sb3,16,sb3,2,6,1,4,1,1,3,1,9,5,3,4,1,0,0,1,5,2,5,1,5,1,1,5,3,2,1,0,0,0,3,3,0,0.642857
8sb3,2.0,7.0,6.0,6.0,6.0,5.0,4.0,1.0,1.0,3.0,3.0,3.0,4.0,6.0,8.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,no response,The house will be sold and the new owner may d...,sb3,8sb3,8,sb3,2,6,1,4,1,1,3,1,9,5,3,4,1,0,0,1,5,2,5,1,5,1,1,5,3,2,1,0,0,0,3,3,0,0.642857


## Final Cleanup

In [ ]:
# build final dataset

final_dataset = all[['respondent','id','archetype_x','narr-age', 'narr-family type',
                      'narr-duration', 'narr-spending',
                      'narr-env', 'narr-income', 'narr-mortgage percent', 'narr-credit',
                      'prm-house sqft', 'prm-lot sqft', 'prm-house age', 'prm-step',
                      'prm-suite', 'prm-reno_plan', 'prm-wall_type', 'prm-wall_age',
                      'prm-window_type', 'prm-window_age', 'prm-roof_type', 'prm-roof_age',
                      'prm-heat_eff', 'prm-heat_fuel', 'prm-heat_age', 'prm-dhw_eff',
                      'prm-dhw_fuel', 'prm-dhw_age', 'prm-cool_eff', 'prm-cool_fuel',
                      'prm-cool_age', 'prm-leakage', 'prm-maintenance', 'prm-asbestos',
                      'prm_derived_FAR', 'expqual-field', 'expqual-field_years',
                      'expqual-step_familiar', 'expqual-market_familiar', 'resp-likert_step2',
                      'resp-likert_step3', 'resp-likert_step4', 'resp-likert_step5',
                      'resp-likert_higheff', 'resp-retrofit_teardown_neither',
                      'resp-likert_carbon2050', 'resp-policy1', 'resp-policy2',
                      'resp-policy3', 'resp-confidence', 'binary-policyopt1',
                      'binary-policyopt2', 'binary-policyopt3', 'binary-policyopt4',
                      'binary-policyopt5', 'binary-policyopt6', 'binary-policyopt7',
                      'binary-policyopt8', 'binary-policyopt9', 'binary-policyopt10',
                      'binary-policyopt11', 'optimism_int',
                      'resp-influential_param','resp-additional_info']]
final_dataset = final_dataset.rename(columns={"archetype_x": "archetype"})

In [ ]:
# just need to clean up the integers before exporting

intcols = ['respondent','archetype','narr-age', 'narr-family type',
            'narr-duration', 'narr-spending',
            'narr-env', 'narr-income', 'narr-mortgage percent', 'narr-credit',
            'prm-house sqft', 'prm-lot sqft', 'prm-house age', 'prm-step',
            'prm-suite', 'prm-reno_plan', 'prm-wall_type', 'prm-wall_age',
            'prm-window_type', 'prm-window_age', 'prm-roof_type', 'prm-roof_age',
            'prm-heat_eff', 'prm-heat_fuel', 'prm-heat_age', 'prm-dhw_eff',
            'prm-dhw_fuel', 'prm-dhw_age', 'prm-cool_eff', 'prm-cool_fuel',
            'prm-cool_age', 'prm-leakage', 'prm-maintenance', 'prm-asbestos',
            'expqual-field', 'expqual-field_years',
            'expqual-step_familiar', 'expqual-market_familiar', 'resp-likert_step2',
            'resp-likert_step3', 'resp-likert_step4', 'resp-likert_step5',
            'resp-likert_higheff', 'resp-retrofit_teardown_neither',
            'resp-likert_carbon2050', 'resp-policy1', 'resp-policy2',
            'resp-policy3', 'resp-confidence', 'binary-policyopt1',
            'binary-policyopt2', 'binary-policyopt3', 'binary-policyopt4',
            'binary-policyopt5', 'binary-policyopt6', 'binary-policyopt7',
            'binary-policyopt8', 'binary-policyopt9', 'binary-policyopt10',
            'binary-policyopt11', 'optimism_int']
floatcols = ['prm_derived_FAR']
strcols = ['id','resp-influential_param','resp-additional_info']
final_dataset[intcols] = final_dataset[intcols].astype(int)


In [ ]:
# Download

final_dataset.to_csv('survey_analysis_dataset.csv')
files.download('survey_analysis_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>